In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
from pso import ParticleSwarmOptimizedClustering
from particle import quantization_error, calc_sse
from utils import normalize
from kmeans import KMeans
from sklearn.metrics import silhouette_score

In [10]:
data = pd.read_csv('./data/heart_processed.txt', sep=' ', header=None)
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,1
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


In [11]:
x = data.drop([13], axis=1)
x = x.values
x = normalize(x)
x

array([[0.70833333, 1.        , 0.        , ..., 1.        , 0.        ,
        0.85714286],
       [0.79166667, 1.        , 1.        , ..., 0.5       , 1.        ,
        0.42857143],
       [0.79166667, 1.        , 1.        , ..., 0.5       , 0.66666667,
        1.        ],
       ...,
       [0.58333333, 1.        , 1.        , ..., 0.5       , 0.33333333,
        1.        ],
       [0.58333333, 0.        , 0.33333333, ..., 0.5       , 0.33333333,
        0.42857143],
       [0.1875    , 1.        , 0.66666667, ..., 0.        , 0.        ,
        0.42857143]])

# K-Means

In [12]:
kmeans = KMeans(n_cluster=2, init_pp=False, seed=2018)
kmeans.fit(x)

In [13]:
predicted_kmeans = kmeans.predict(x)
print('Silhouette:', silhouette_score(x, predicted_kmeans))
print('SSE:', kmeans.SSE)
print('Quantization:', quantization_error(centroids=kmeans.centroid, data=x, labels=predicted_kmeans))

Silhouette: 0.1493071878000349
SSE: 343.7566715135276
Quantization: 1.0443919297332505


In [15]:
kmeans2 = KMeans(n_cluster=2, init_pp=True, seed=2018)
kmeans2.fit(x)
predicted_kmeans2 = kmeans2.predict(x)
print('Silhouette:', silhouette_score(x, predicted_kmeans))
print('SSE:', kmeans2.SSE)
print('Quantization:', quantization_error(centroids=kmeans2.centroid, data=x, labels=predicted_kmeans2))

Silhouette: 0.1493071878000349
SSE: 324.86578873437315
Quantization: 1.0144138871665043


# PSO

In [16]:
pso = ParticleSwarmOptimizedClustering(
        n_cluster=2, n_particles=10, data=x,dump_w=0.99,c1=2,c2=2,w=1, hybrid=True, max_iter=2000, print_debug=50)

In [17]:
hist = pso.run()

Initial global best score 1.0008511461371405
Iteration 0001/2000 current gbest score 1.000851146137140546


d:\study\2SIQ3\OPTIM\Partitioning_Clustering_using_ACO_metaheuristic\particle.py:14: RuntimeWarning: invalid value encountered in double_scalars
  dist /= len(idx)


Iteration 0051/2000 current gbest score 0.999965017550151325
Iteration 0101/2000 current gbest score 0.998939157891765017
Iteration 0151/2000 current gbest score 0.998901929686583867
Iteration 0201/2000 current gbest score 0.998901758303202936
Iteration 0251/2000 current gbest score 0.998901758251614869
Iteration 0301/2000 current gbest score 0.998901758251486971
Iteration 0351/2000 current gbest score 0.998901758251486749
Iteration 0401/2000 current gbest score 0.998901758251486749
Iteration 0451/2000 current gbest score 0.998901758251486749
Iteration 0501/2000 current gbest score 0.998901758251486749
Iteration 0551/2000 current gbest score 0.998901758251486749
Iteration 0601/2000 current gbest score 0.998901758251486749
Iteration 0651/2000 current gbest score 0.998901758251486749
Iteration 0701/2000 current gbest score 0.998901758251486749
Iteration 0751/2000 current gbest score 0.998901758251486749
Iteration 0801/2000 current gbest score 0.998901758251486749
Iteration 0851/2000 curr

In [18]:
pso_kmeans = KMeans(n_cluster=2, init_pp=False, seed=2018)

In [19]:
pso_kmeans.centroid = pso.gbest_centroids.copy()
pso_kmeans.centroid

array([[0.54159854, 0.81962358, 0.92327019, 0.36500171, 0.29403214,
        0.12859444, 0.55400579, 0.49094419, 1.01342457, 0.24649123,
        0.41655418, 0.29332361, 0.81137532],
       [0.49449283, 0.64906291, 0.61599591, 0.34141059, 0.27589898,
        0.12958011, 0.42637593, 0.65038961, 0.0132823 , 0.11612488,
        0.22459649, 0.16332468, 0.61015622]])

In [20]:
predicted_pso = pso_kmeans.predict(x)
print('Silhouette:', silhouette_score(x, predicted_pso))
print('SSE:', calc_sse(centroids=pso.gbest_centroids, data=x, labels=predicted_pso))
print('Quantization:', pso.gbest_score)

Silhouette: 0.21415960170661166
SSE: 322.4571214962663
Quantization: 0.9989017582514867


# Repeated Test

### K-Means++

In [21]:
kmeanspp = {
    'silhouette': [],
    'sse' : [],
    'quantization' : [],
}
for _ in range(20):
    kmean_rep = KMeans(n_cluster=2, init_pp=True)
    kmean_rep.fit(x)
    predicted_kmean_rep = kmean_rep.predict(x)
    #print(predicted_kmean_rep)
    silhouette = silhouette_score(x, predicted_kmean_rep)
    sse = kmean_rep.SSE
    quantization = quantization_error(centroids=kmean_rep.centroid, data=x, labels=predicted_kmean_rep)
    kmeanspp['silhouette'].append(silhouette)
    kmeanspp['sse'].append(sse)
    kmeanspp['quantization'].append(quantization)

### PSO 

In [22]:
%%time
pso_plain = {
    'silhouette': [],
    'sse' : [],
    'quantization' : [],
}
for _ in range(20):
    pso_rep = ParticleSwarmOptimizedClustering(
        n_cluster=2, n_particles=10, data=x,dump_w=0.99,c1=2,c2=2,w=1, hybrid=False, max_iter=2000, print_debug=2000)
    pso_rep.run()
    pso_kmeans = KMeans(n_cluster=2, init_pp=False, seed=2018)
    pso_kmeans.centroid = pso_rep.gbest_centroids.copy()
    predicted_pso_rep = pso_kmeans.predict(x)
    
    silhouette = silhouette_score(x, predicted_pso_rep)
    sse = calc_sse(centroids=pso_rep.gbest_centroids, data=x, labels=predicted_pso_rep)
    quantization = pso_rep.gbest_score
    pso_plain['silhouette'].append(silhouette)
    pso_plain['sse'].append(sse)
    pso_plain['quantization'].append(quantization)

Initial global best score 1.282938464349118
Iteration 0001/2000 current gbest score 1.158293117580312170


d:\study\2SIQ3\OPTIM\Partitioning_Clustering_using_ACO_metaheuristic\particle.py:14: RuntimeWarning: invalid value encountered in double_scalars
  dist /= len(idx)


Finish with gbest score 1.024463416277286765
Initial global best score 1.246677277835623
Iteration 0001/2000 current gbest score 1.156871655412359701


d:\study\2SIQ3\OPTIM\Partitioning_Clustering_using_ACO_metaheuristic\particle.py:14: RuntimeWarning: invalid value encountered in double_scalars
  dist /= len(idx)


Finish with gbest score 1.057335058371712044
Initial global best score 1.2424063015333595
Iteration 0001/2000 current gbest score 1.180450443304050356


...

Finish with gbest score 1.045226603103827756
Initial global best score 1.272941285858539
Iteration 0001/2000 current gbest score 1.123353783705678399


d:\study\2SIQ3\OPTIM\Partitioning_Clustering_using_ACO_metaheuristic\particle.py:14: RuntimeWarning: invalid value encountered in double_scalars
  dist /= len(idx)


Finish with gbest score 1.009977410675027221
Initial global best score 1.2186982933202226
Iteration 0001/2000 current gbest score 1.218698293320222614


d:\study\2SIQ3\OPTIM\Partitioning_Clustering_using_ACO_metaheuristic\particle.py:14: RuntimeWarning: invalid value encountered in double_scalars
  dist /= len(idx)


Finish with gbest score 1.007190706267411473
Wall time: 4min 10s


### PSO Hybrid

In [23]:
%%time
pso_hybrid = {
    'silhouette': [],
    'sse' : [],
    'quantization' : [],
}
for _ in range(20):
    pso_rep = ParticleSwarmOptimizedClustering(
        n_cluster=2, n_particles=10, data=x,dump_w=0.99,c1=2,c2=2,w=1, hybrid=True, max_iter=2000, print_debug=2000)
    pso_rep.run()
    pso_kmeans = KMeans(n_cluster=2, init_pp=False, seed=2018)
    pso_kmeans.centroid = pso_rep.gbest_centroids.copy()
    predicted_pso_rep = pso_kmeans.predict(x)
    
    silhouette = silhouette_score(x, predicted_pso_rep)
    sse = calc_sse(centroids=pso_rep.gbest_centroids, data=x, labels=predicted_pso_rep)
    quantization = pso_rep.gbest_score
    pso_hybrid['silhouette'].append(silhouette)
    pso_hybrid['sse'].append(sse)
    pso_hybrid['quantization'].append(quantization)

Initial global best score 1.0008511461371405
Iteration 0001/2000 current gbest score 1.000851146137140546


d:\study\2SIQ3\OPTIM\Partitioning_Clustering_using_ACO_metaheuristic\particle.py:14: RuntimeWarning: invalid value encountered in double_scalars
  dist /= len(idx)


Finish with gbest score 1.000067721314768621
Initial global best score 1.0008511461371405
Iteration 0001/2000 current gbest score 1.000851146137140546


d:\study\2SIQ3\OPTIM\Partitioning_Clustering_using_ACO_metaheuristic\particle.py:14: RuntimeWarning: invalid value encountered in double_scalars
  dist /= len(idx)


Finish with gbest score 0.999368979747785602
Initial global best score 1.0008511461371405
Iteration 0001/2000 current gbest score 1.000851146137140546


...

Finish with gbest score 0.999920168774673934
Initial global best score 1.0048541984610915
Iteration 0001/2000 current gbest score 1.004854198461091475


d:\study\2SIQ3\OPTIM\Partitioning_Clustering_using_ACO_metaheuristic\particle.py:14: RuntimeWarning: invalid value encountered in double_scalars
  dist /= len(idx)


Finish with gbest score 1.003640363530258650
Initial global best score 1.0008511461371405
Iteration 0001/2000 current gbest score 1.000851146137140546


d:\study\2SIQ3\OPTIM\Partitioning_Clustering_using_ACO_metaheuristic\particle.py:14: RuntimeWarning: invalid value encountered in double_scalars
  dist /= len(idx)


Finish with gbest score 0.999358656318809380
Wall time: 1min 45s


# Comparison

In [24]:
benchmark = {
    'method' : ['K-Means++', 'PSO', 'PSO Hybrid'],
    'sse_mean' : [
        np.around(np.mean(kmeanspp['sse']), decimals=10),
        np.around(np.mean(pso_plain['sse']), decimals=10),
        np.around(np.mean(pso_hybrid['sse']), decimals=10),
    ],
    'sse_stdev' : [
        np.around(np.std(kmeanspp['sse']), decimals=10),
        np.around(np.std(pso_plain['sse']), decimals=10),
        np.around(np.std(pso_hybrid['sse']), decimals=10),
    ],
    'silhouette_mean' : [
        np.around(np.mean(kmeanspp['silhouette']), decimals=10),
        np.around(np.mean(pso_plain['silhouette']), decimals=10),
        np.around(np.mean(pso_hybrid['silhouette']), decimals=10),
    ],
    'silhouette_stdev' : [
        np.around(np.std(kmeanspp['silhouette']), decimals=10),
        np.around(np.std(pso_plain['silhouette']), decimals=10),
        np.around(np.std(pso_hybrid['silhouette']), decimals=10),
    ],
    'quantization_mean' : [
        np.around(np.mean(kmeanspp['quantization']), decimals=10),
        np.around(np.mean(pso_plain['quantization']), decimals=10),
        np.around(np.mean(pso_hybrid['quantization']), decimals=10),
    ],
    'quantization_stdev' : [
        np.around(np.std(kmeanspp['quantization']), decimals=10),
        np.around(np.std(pso_plain['quantization']), decimals=10),
        np.around(np.std(pso_hybrid['quantization']), decimals=10),
    ],
}

In [25]:
benchmark

{'method': ['K-Means++', 'PSO', 'PSO Hybrid'],
 'sse_mean': [328.6670608596, 391.6973940503, 323.396516429],
 'sse_stdev': [13.8896666801, 36.1625748871, 3.0138386068],
 'silhouette_mean': [0.2009297849, 0.1343657739, 0.20871241],
 'silhouette_stdev': [0.0124132872, 0.0516522078, 0.0094420556],
 'quantization_mean': [1.0194684532, 1.0123951504, 1.0020165163],
 'quantization_stdev': [0.0297859789, 0.0508387423, 0.004504653]}

In [26]:
benchmark_df = pd.DataFrame.from_dict(benchmark)
benchmark_df

,method,sse_mean,sse_stdev,silhouette_mean,silhouette_stdev,quantization_mean,quantization_stdev
0,K-Means++,328.667061,13.889667,0.200930,0.012413,1.019468,0.029786
1,PSO,391.697394,36.162575,0.134366,0.051652,1.012395,0.050839
2,PSO Hybrid,323.396516,3.013839,0.208712,0.009442,1.002017,0.004505


In [27]:
benchmark_df.to_excel('benchmark_heart_res.xlsx', index=False)

In [28]:
benchmark_df.to_csv('benchmark_heart_res.csv', index=False)